# SSA Popular Baby Names

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [25]:
import pandas as pd
import altair as alt
import datetime as dt
import glob
import os
import requests

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.options.display.max_colwidth = None

In [4]:
today = dt.datetime.today().strftime("%Y-%m-%d")

---

### Download [latest batch](https://www.ssa.gov/oact/babynames/limits.html) of names data from SSA

In [5]:
!curl -s 'https://www.ssa.gov/oact/babynames/names.zip' --output data/raw/names.zip

In [6]:
!unzip -o -q 'data/raw/names.zip' -d 'data/raw/years'

In [7]:
!cat data/raw/years/*.txt > 'data/raw/all.txt'

---

In [8]:
path = "data/raw/years"
all_files = glob.glob(os.path.join(path, "*.txt"))

df_from_each_file = (
    pd.read_csv(
        f,
        encoding="ISO-8859-1",
        header=None,
        sep=",",
        low_memory=False,
        names=["name", "sex", "count"],
    ).assign(year=f)
    for f in all_files
)
names = pd.concat(df_from_each_file, ignore_index=True)

In [9]:
names["year"] = (
    names["year"]
    .str.replace("data/raw/years/yob", "", regex=True)
    .str.replace(".txt", "", regex=True)
)

In [10]:
names.head(15)

,name,sex,count,year
0,Emily,F,25959,2000
1,Hannah,F,23086,2000
2,Madison,F,19968,2000
3,Ashley,F,17998,2000
4,Sarah,F,17713,2000
5,Alexis,F,17631,2000
6,Samantha,F,17264,2000
7,Jessica,F,15711,2000
8,Elizabeth,F,15116,2000
9,Taylor,F,15079,2000


### Limit names since 1950

In [11]:
names[names["year"].astype(int) >= 1950].tail()

,name,sex,count,year
2117214,Zyheem,M,5,2019
2117215,Zykel,M,5,2019
2117216,Zyking,M,5,2019
2117217,Zyn,M,5,2019
2117218,Zyran,M,5,2019


---

### Get birth totals for normalization

In [12]:
url = "https://www.ssa.gov/oact/babynames/numberUSbirths.html"

In [13]:
births = pd.read_html(url)[0]

In [14]:
births = births.rename(
    columns={
        "Year of  birth": "year",
        "Male": "male",
        "Female": "female",
        "Total": "total",
    },
)

In [15]:
births["year"] = births.year.astype(str)

In [16]:
births

,year,male,female,total
0,1880,118399,97605,216004
1,1881,108276,98855,207131
2,1882,122031,115694,237725
3,1883,112475,120060,232535
4,1884,122738,137585,260323
5,1885,115945,141947,257892
6,1886,119041,153734,272775
7,1887,109311,155423,264734
8,1888,129907,189445,319352
9,1889,119032,189218,308250


In [17]:
births.head(15)

,year,male,female,total
0,1880,118399,97605,216004
1,1881,108276,98855,207131
2,1882,122031,115694,237725
3,1883,112475,120060,232535
4,1884,122738,137585,260323
5,1885,115945,141947,257892
6,1886,119041,153734,272775
7,1887,109311,155423,264734
8,1888,129907,189445,319352
9,1889,119032,189218,308250


--- 

### Merge the dataframes together

In [18]:
df = pd.merge(names, births, on="year")

In [19]:
df.head()

,name,sex,count,year,male,female,total
0,Emily,F,25959,2000,2088325,1995743,4084068
1,Hannah,F,23086,2000,2088325,1995743,4084068
2,Madison,F,19968,2000,2088325,1995743,4084068
3,Ashley,F,17998,2000,2088325,1995743,4084068
4,Sarah,F,17713,2000,2088325,1995743,4084068


### Calculate rate

In [20]:
df["prop"] = df["count"] * 1.0 / df["total"]
df["fprop"] = df[df["sex"] == "F"]["count"] / df[df["sex"] == "F"]["female"]
df["mprop"] = df[df["sex"] == "M"]["count"] / df[df["sex"] == "M"]["male"]

In [ ]:
df.head()

---

#### Normalized for TikTok vid

In [ ]:
# Load the data
ssa_applications_url = (
    "https://stilesdata.com/babynames/us_ssa_applications_birth_years.json"
)
population_url = "https://stilesdata.com/babynames/us_population_years_census.json"

ssa_applications = pd.read_json(ssa_applications_url)
population = pd.read_json(population_url)

# Convert the 'year' column to integers
ssa_applications["year"] = ssa_applications["year"].astype(int)
population["year"] = population["year"].astype(int)

# Filter the data to start from the year 1900
ssa_applications = ssa_applications[ssa_applications["year"] >= 1900]

# Merge the two dataframes on the 'year' column
merged_data = pd.merge(ssa_applications, population, on="year", how="left")

# Normalize the SSA applications by population per 1 million
merged_data["male_per_million"] = (
    merged_data["male"] / merged_data["population"]
) * 1e6
merged_data["female_per_million"] = (
    merged_data["female"] / merged_data["population"]
) * 1e6

# Fill any NaN values resulting from the merge with 0
merged_data.fillna(0, inplace=True)

# Save the processed data to a JSON file
normalized_data = merged_data[["year", "male_per_million", "female_per_million"]].query(
    "year > 1899 and year < 2023"
)
normalized_data.to_json(
    "data/processed/normalized_ssa_applications_per_million.json", orient="records"
)

---

### Export

In [ ]:
df.to_csv("data/processed/names_births.csv", index=False)

In [ ]:
births.to_json(
    "data/processed/us_ssa_applications_birth_years.json",
    indent=4,
    orient="records",
    lines=False,
)

In [55]:
pop_census.to_json(
    "data/processed/us_population_years_census.json",
    indent=4,
    orient="records",
    lines=False,
)